## Torch test

In [1]:
# import os

# os.environ["OMP_NUM_THREADS"] = "1"
import torch
import emcee
import pandas as pd
from scipy.spatial import KDTree
from multiprocessing import Pool

# Modify the code to set the device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def interpolated_mag(theta, mag_table):
    Teff, log_g, Z, d = theta

    # Move relevant data to CPU for KDTree processing
    kdtree = KDTree(mag_table[['Teff', 'log_g', 'Z']])
    dist, points = kdtree.query(theta[:3], 1)
    
    # Move the result back to GPU if necessary
    mag = torch.tensor(mag_table.iloc[points][['F115W', 'F150W', 'F277W', 'F444W']].values, device=device)
    
    # Convert fluxes to magnitudes, mag_table is AB magnitude at 10 pc
    Mag = mag + 5 * torch.log10(torch.tensor(d) / 10)
    
    return Mag

def log_likelihood(theta, mag_table, y, yerr):
    model_m = interpolated_mag(theta, mag_table)
    return -0.5 * torch.sum(((y - model_m) / yerr) ** 2)

def log_prior(theta):
    Teff, log_g, Z, d = theta
    if 400 < Teff < 2400 and 2.9 < log_g < 5.6 and 0.4 < Z < 1.6 and 9 < d < 4000:
        return torch.tensor(0.0, device=device)
    return torch.tensor(-float('inf'), device=device)

def log_probability(theta, mag_table, y, yerr):
    lp = log_prior(theta)
    if not torch.isfinite(lp):  # Ensure prior is finite
        return -float('inf')
    return (lp + log_likelihood(theta, mag_table, y, yerr)).item()

# Ensure that your tensors are on the correct device
theta = torch.tensor([1000, 4, 1, 1500], device=device)
coefficients = torch.tensor([500, 3, 1, 3000], device=device)


In [2]:
n=1

mag_table = pd.read_feather('interpolated_ABmag.feather')
observation = pd.read_csv('final.csv')

# Convert observation data to PyTorch tensors
magnitudes = torch.tensor(observation[['MAG_AUTO_F115W', 'MAG_AUTO_F150W', 'MAG_AUTO_F277W', 'MAG_AUTO_F444W']].values, device=device)
magerr = torch.tensor(observation[['MAGERR_AUTO_F115W', 'MAGERR_AUTO_F150W', 'MAGERR_AUTO_F277W', 'MAGERR_AUTO_F444W']].values, device=device)
Mobs, Merr = lambda x: magnitudes[x-1], lambda x: magerr[x-1]

# Set the initial guess and coefficients as PyTorch tensors
theta = torch.tensor([1000, 4, 1, 1500], device=device)
coefficients = torch.tensor([500, 3, 1, 2000], device=device)

# Setup walkers
pos = theta + coefficients * torch.randn((160, 4), device=device)
nwalkers, ndim = pos.shape

# Convert the walker positions to numpy for use with emcee
pos_numpy = pos.cpu().numpy()

# # Run the MCMC fitting process
sampler = emcee.EnsembleSampler(
    nwalkers, ndim, log_probability, args=(mag_table, Mobs(n), Merr(n))
)

# sun the mcmc fitting
# with Pool() as pool:
#     sampler = emcee.EnsembleSampler(
#         nwalkers, ndim, log_probability, pool=pool, args=(mag_table, Mobs(n), Merr(n))
#     ) 
#     sampler.run_mcmc(pos, 5000, progress=True)

In [3]:
sampler.run_mcmc(pos_numpy, 5000, progress=True)

# Convert sampler chain back to PyTorch tensors if further processing on GPU is required
# samples = torch.tensor(sampler.chain, device=device)

  0%|          | 0/5000 [00:00<?, ?it/s]/home/amos/miniconda3/envs/Eva00/lib/python3.12/site-packages/emcee/moves/red_blue.py:99: RuntimeWarning: invalid value encountered in scalar subtract
  lnpdiff = f + nlp - state.log_prob[j]
/home/amos/miniconda3/envs/Eva00/lib/python3.12/site-packages/emcee/moves/red_blue.py:99: RuntimeWarning: invalid value encountered in scalar subtract
  lnpdiff = f + nlp - state.log_prob[j]
  0%|          | 1/5000 [00:00<26:43,  3.12it/s]/home/amos/miniconda3/envs/Eva00/lib/python3.12/site-packages/emcee/moves/red_blue.py:99: RuntimeWarning: invalid value encountered in scalar subtract
  lnpdiff = f + nlp - state.log_prob[j]
/home/amos/miniconda3/envs/Eva00/lib/python3.12/site-packages/emcee/moves/red_blue.py:99: RuntimeWarning: invalid value encountered in scalar subtract
  lnpdiff = f + nlp - state.log_prob[j]
  0%|          | 2/5000 [00:00<26:58,  3.09it/s]/home/amos/miniconda3/envs/Eva00/lib/python3.12/site-packages/emcee/moves/red_blue.py:99: RuntimeWar

emcee: Exception while calling your likelihood function:
  params: [8.85634702e+02 4.41551561e+00 6.68871848e-01 6.56605772e+02]
  args: (          Teff  log_g    Z      F115W      F150W      F277W      F444W  \
0        400.0    3.0  0.5  22.456816  23.941816  23.846566  17.565566   
1        400.0    3.0  0.6  22.353566  23.971366  23.875616  17.571766   
2        400.0    3.0  0.7  22.250316  24.000916  23.904666  17.577966   
3        400.0    3.0  0.8  22.147066  24.030466  23.933716  17.584166   
4        400.0    3.0  0.9  22.043816  24.060016  23.962766  17.590366   
...        ...    ...  ...        ...        ...        ...        ...   
114681  2400.0    5.5  1.1  12.723066  12.634866  13.400016  13.365116   
114682  2400.0    5.5  1.2  12.697816  12.627166  13.372966  13.346166   
114683  2400.0    5.5  1.3  12.672566  12.619466  13.345916  13.327216   
114684  2400.0    5.5  1.4  12.647316  12.611766  13.318866  13.308266   
114685  2400.0    5.5  1.5  12.622066  12.604066

KeyboardInterrupt: 

## cuda test

In [1]:
# import os

# os.environ["OMP_NUM_THREADS"] = "1"

# import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
# import argparse
# import corner
import emcee
# pytorch 2.4.0 for cuda 11.8
# import torch
import cupy as cp

# from scipy.interpolate import LinearNDInterpolator
from scipy.spatial import KDTree
from multiprocessing import Pool
from multiprocessing import set_start_method


In [2]:
def interpolated_mag(theta, mag_table):
    Teff, log_g, Z, d = theta
    
    # Find the nearest point in the grid
    kdtree = KDTree(mag_table[['Teff', 'log_g', 'Z']])
    dist, points = kdtree.query(theta[:3], 1)
    
    mag = mag_table.iloc[points][['F115W', 'F150W', 'F277W', 'F444W']].values
    
    # Convert fluxes to magnitudes, mag_table is AB magnitude at 10 pc
    Mag = mag + 5 * np.log10(d / 10)
    
    return Mag

def log_likelihood(theta, mag_table, y, yerr):
    model_m = interpolated_mag(theta, mag_table)
    return -0.5 * cp.sum(((cp.array(y) - cp.array(model_m)) / cp.array(yerr)) ** 2)  # likelihood function

def log_prior(theta):
    Teff, log_g, Z, d = theta
    if 400 < Teff < 2400 and 2.9 < log_g < 5.6 and 0.4 < Z < 1.6 and 9 < d < 3000:  # set a reasonable range of parameters
        return 0.0
    return -cp.inf

def log_probability(theta, mag_table, y, yerr):
    lp = log_prior(theta)
    if not cp.isfinite(lp):  # if the parameter is not reasonable, return -inf (0 possibility)
        return -cp.inf
    return lp + log_likelihood(theta, mag_table, y, yerr)


In [4]:
mag_table = pd.read_feather('interpolated_ABmag.feather')
observation = pd.read_csv('final.csv')

magnitudes = observation[['MAG_AUTO_F115W', 'MAG_AUTO_F150W', 'MAG_AUTO_F277W', 'MAG_AUTO_F444W']].values
magerr = observation[['MAGERR_AUTO_F115W', 'MAGERR_AUTO_F150W', 'MAGERR_AUTO_F277W', 'MAGERR_AUTO_F444W']].values
Mobs, Merr = lambda x: magnitudes[x-1], lambda x: magerr[x-1]

n = 1

# set initial guess 
theta = 1000, 4, 1, 1500
coefficients = np.array([500, 3, 1, 2000])

# setup walkers
pos = theta + coefficients * np.random.randn(32, 4)
nwalkers, ndim = pos.shape

# sun the mcmc fitting
# with Pool() as pool:
#     sampler = emcee.EnsembleSampler(
#         nwalkers, ndim, log_probability, pool=pool, args=(mag_table, Mobs(n), Merr(n))
#     ) 
#     sampler.run_mcmc(pos, 5000, progress=True)

sampler = emcee.EnsembleSampler(
    nwalkers, ndim, log_probability, args=(mag_table, Mobs(n), Merr(n))
) 

In [5]:
sampler.run_mcmc(pos, 5000, progress=True)

emcee: Exception while calling your likelihood function:
  params: [1317.48593725    7.57313632    1.47663511 1013.97238156]
  args: (          Teff  log_g    Z      F115W      F150W      F277W      F444W  \
0        400.0    3.0  0.5  22.456816  23.941816  23.846566  17.565566   
1        400.0    3.0  0.6  22.353566  23.971366  23.875616  17.571766   
2        400.0    3.0  0.7  22.250316  24.000916  23.904666  17.577966   
3        400.0    3.0  0.8  22.147066  24.030466  23.933716  17.584166   
4        400.0    3.0  0.9  22.043816  24.060016  23.962766  17.590366   
...        ...    ...  ...        ...        ...        ...        ...   
114681  2400.0    5.5  1.1  12.723066  12.634866  13.400016  13.365116   
114682  2400.0    5.5  1.2  12.697816  12.627166  13.372966  13.346166   
114683  2400.0    5.5  1.3  12.672566  12.619466  13.345916  13.327216   
114684  2400.0    5.5  1.4  12.647316  12.611766  13.318866  13.308266   
114685  2400.0    5.5  1.5  12.622066  12.604066  13

Traceback (most recent call last):
  File "cupy_backends/cuda/_softlink.pyx", line 25, in cupy_backends.cuda._softlink.SoftLink.__init__
  File "/home/amos/miniconda3/envs/Eva00/lib/python3.12/ctypes/__init__.py", line 379, in __init__
    self._handle = _dlopen(self._name, mode)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^
OSError: libnvrtc.so.11.2: cannot open shared object file: No such file or directory

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/amos/miniconda3/envs/Eva00/lib/python3.12/site-packages/emcee/ensemble.py", line 640, in __call__
    return self.f(x, *self.args, **self.kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_32641/2974881562.py", line 27, in log_probability
    if not cp.isfinite(lp):  # if the parameter is not reasonable, return -inf (0 possibility)
           ^^^^^^^^^^^^^^^
  File "cupy/_core/_kernel.pyx", line 1375, in cupy._core._kernel.ufunc.__call__

RuntimeError: CuPy failed to load libnvrtc.so.11.2: OSError: libnvrtc.so.11.2: cannot open shared object file: No such file or directory

## normal cpu run

In [1]:
import os

os.environ["OMP_NUM_THREADS"] = "1"

# import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
# import argparse
# import corner
import emcee

from scipy.spatial import KDTree
from multiprocessing import Pool

def interpolated_mag(theta, mag_table):
    Teff, log_g, Z, d = theta
    
    # Find the nearest point in the grid
    kdtree = KDTree(mag_table[['Teff', 'log_g', 'Z']])
    dist, points = kdtree.query(theta[:3], 1)
    
    mag = mag_table.iloc[points][['F115W', 'F150W', 'F277W', 'F444W']].values
    
    # Convert fluxes to magnitudes, mag_table is AB magnitude at 10 pc
    Mag = mag + 5 * np.log10(d / 10)
    
    return Mag

def log_likelihood(theta, mag_table, y, yerr):
    model_m = interpolated_mag(theta, mag_table)
    return -0.5 * np.sum(((np.array(y) - np.array(model_m)) / np.array(yerr)) ** 2)  # likelihood function

def log_prior(theta):
    Teff, log_g, Z, d = theta
    if 400 < Teff < 2400 and 2.9 < log_g < 5.6 and 0.4 < Z < 1.6 and 9 < d < 4000:  # set a reasonable range of parameters
        return 0.0
    return -np.inf

def log_probability(theta, mag_table, y, yerr):
    lp = log_prior(theta)
    if not np.isfinite(lp):  # if the parameter is not reasonable, return -inf (0 possibility)
        return -np.inf
    return lp + log_likelihood(theta, mag_table, y, yerr)

# file_dir = '/cluster/home/yuanchen/JWDT/BD_mcmc/'

mag_table = pd.read_feather('interpolated_ABmag.feather')
observation = pd.read_csv('final.csv')

magnitudes = observation[['MAG_AUTO_F115W', 'MAG_AUTO_F150W', 'MAG_AUTO_F277W', 'MAG_AUTO_F444W']].values
magerr = observation[['MAGERR_AUTO_F115W', 'MAGERR_AUTO_F150W', 'MAGERR_AUTO_F277W', 'MAGERR_AUTO_F444W']].values
Mobs, Merr = lambda x: magnitudes[x-1], lambda x: magerr[x-1]

n = 20

# set initial guess 
theta = 1000, 4, 1, 1500
coefficients = np.array([500, 3, 1, 2000])

# setup walkers
pos = theta + coefficients * np.random.randn(160, 4)
nwalkers, ndim = pos.shape

# sun the mcmc fitting
with Pool() as pool:
    sampler = emcee.EnsembleSampler(
        nwalkers, ndim, log_probability, pool=pool, args=(mag_table, Mobs(n), Merr(n))
    ) 
    sampler.run_mcmc(pos, 5000, progress=True)
    
# sampler = emcee.EnsembleSampler(
#         nwalkers, ndim, log_probability, args=(mag_table, Mobs(n), Merr(n))
#     ) 
# sampler.run_mcmc(pos, 5000, progress=True)

# refine results and drop data affected by initial guess
flat_samples = sampler.get_chain(discard=600, thin=10, flat=True)

data = pd.DataFrame(flat_samples, columns=['Teff', 'log_g', '[M/H]', 'd'])
if n<9:
    data.to_csv(file_dir+f'results/BD0{n}_mcmc.csv')
else:
    data.to_csv(file_dir+f'results/BD{n}_mcmc.csv')

  0%|          | 0/5000 [00:00<?, ?it/s]/home/amos/miniconda3/envs/Eva00/lib/python3.12/site-packages/emcee/moves/red_blue.py:99: RuntimeWarning: invalid value encountered in scalar subtract
  lnpdiff = f + nlp - state.log_prob[j]
  2%|▏         | 81/5000 [01:13<1:14:45,  1.10it/s]


KeyboardInterrupt: 